In [10]:
# imports
import cv2
import os
import numpy as np
import math
from dataclasses import dataclass,fields
from utils import *

In [11]:
# TODO:convert_video_to_frames
"""
params : video name 
return : create output directory include the frames of this video 
"""
def convert_video_to_frames(video_name):
    video_path = "../Dataset/"+video_name
    output_dir = "../Dataset/frames_"+video_name
    sampling_rate = 60
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    for frame_idx in range(0, total_frames, sampling_rate):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if ret:
            output_path = os.path.join(output_dir, "frame_{:06d}.jpg".format(frame_idx))
            cv2.imwrite(output_path, frame)
    cap.release()
    return 

In [12]:
# TODO: create classes 
"""
desc: Difference between pixel hue,saturation,luma,edges values of adjacent frames.
"""
from typing import Optional
@dataclass
class delta_feature:
    # by default set all weights to 1
    delta_hue: float = 1.0
    delta_sat: float = 1.0
    delta_lum: float = 1.0
    # TODO: have bigger values that other features,detection threshold may need to be adjusted
    delta_edges: float = 1.0  

"""
desc: features calculated for each frame
"""
@dataclass
class frame_data:
    hue: np.ndarray
    sat: np.ndarray
    lum: np.ndarray
    edges: Optional[np.ndarray]
    # TODO: add optical flow

# create defualt weights 
default_delta_feature_weights = delta_feature()
@dataclass
class shot_detector:
    threshold: float = 27.0
    min_scene_len: int = 15
    weights: 'delta_feature' = default_delta_feature_weights
    kernel_size: Optional[int] = None
    last_scene_cut: Optional[int] = None
    last_frame: Optional[frame_data] = None
    frame_score: float=None


In [13]:
"""Detect edges  in the frame 
    params:
        lum:  the luma channel of a frame.
        kernel: kernel size 
    return:
        2D 8-bit image where 255--> edge , 0--> other 
"""
import math
def detect_edges(lum: np.ndarray,kernel:int =None) -> np.ndarray:
    if kernel == None:
        # calculate kernel size  depend on the video reselution
        kernel_size = 4 + round(math.sqrt(lum.shape[1]*lum.shape[0]) / 192)
        if kernel_size % 2 == 0:
            kernel_size += 1
        kernel = np.ones((kernel_size, kernel_size), np.uint8) 
    # Estimate levels for thresholding.
    sigma: float = 1.0 / 3.0
    median = np.median(lum)
    low = int(max(0, (1.0 - sigma) * median))
    high = int(min(255, (1.0 + sigma) * median))
    # Calculate edges using Canny algorithm, and reduce noise by dilating the edges.
    edges = cv2.Canny(lum, low, high)
    return cv2.dilate(edges,kernel)

In [14]:
"""
calculate fram score , to compare with the threshold and know if it is shot boundry or not 
params
 frame_num: index of the frame 
 frame_img: frame itself
"""
def calculate_frame_score(video_param:shot_detector,frame_img: np.ndarray) -> float:
        # TODO: Add option to enable motion estimation
        # convert image into HSV colorspace
        hue, sat, lum = cv2.split(cv2.cvtColor(frame_img, cv2.COLOR_BGR2HSV))
        # calculate edges 
        edges = detect_edges(lum) 
        if video_param.last_frame is None:
            video_param.last_frame = frame_data(hue, sat, lum, edges)
            return 0.0
        score_components = delta_feature(
            delta_hue=mean_pixel_distance(hue, video_param.last_frame.hue),
            delta_sat=mean_pixel_distance(sat, video_param.last_frame.sat),
            delta_lum=mean_pixel_distance(lum, video_param.last_frame.lum),
            delta_edges=(0.0 if edges is None else mean_pixel_distance(
                edges, video_param.last_frame.edges)),
        )
        frame_score: float = (
            sum(getattr(score_components, sco_com.name)* getattr(video_param.weights, wei.name) for (sco_com, wei) in zip(fields(score_components),fields(video_param.weights)))
            / sum(abs(getattr(video_param.weights, weigh.name)) for weigh in fields(video_param.weights)))
        # Store all data required to calculate the next frame's score.
        video_param.last_frame = frame_data(hue, sat, lum, edges)
        return frame_score

In [15]:
"""
return: List of frames where scene cuts have been detected
"""
def process_frame(video_param:shot_detector,frame_num: int, frame_img: np.ndarray):
        if frame_img is None:
            return []
        # initialize the beginning of the frames that contain shot boundery
        if video_param.last_scene_cut is None:
            video_param.last_scene_cut = frame_num
        video_param.frame_score = calculate_frame_score(video_param,frame_img)
        if video_param.frame_score is None:
            return []
        '''
        consider any frame over the threshold a new scene, but only if
        the minimum scene length has been reached (otherwise it is ignored).
        '''
        if video_param.frame_score >= video_param.threshold and (
            (frame_num - video_param.last_scene_cut) >= video_param.min_scene_len):
            video_param.last_scene_cut = frame_num
            # TODO: create average feature vector for these frames
             
            return [frame_num]
        return []

In [23]:

# try 
import cv2
import os
video_path = "../tears_of_steel_1080p.mov"
output_dir = "frames"
sampling_rate = 10
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
cap = cv2.VideoCapture(video_path)
video_try = shot_detector()
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
for frame_idx in range(0, total_frames, sampling_rate):
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
    ret, frame = cap.read()
    if ret:
        output_path = os.path.join(output_dir, "frame_{:06d}.jpg".format(frame_idx))
        cv2.imwrite(output_path, frame)
    print(process_frame(video_try,frame_idx,frame))
cap.release()

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[220]
[]
[240]
[]
[260]
[]
[280]
[]
[]
[]
[]
[330]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[450]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[600]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[970]
[]
[]
[1000]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[1100]
[]
[1120]
[]
[1140]
[]
[1160]
[]
[1180]
[]
[1200]
[]
[1220]
[]
[]
[]
[]
[]
[]
[1290]
[]
[1310]
[]
[]
[]
[]
[1360]
[]
[]
[1390]
[]
[]
[1420]
[]
[1440]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[1550]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[1680]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[1780]
[]
[]
[]
[]
[1830]
[]
[1850]
[]
[]
[1880]
[]
[1900]
[]
[]
[]
[]
[]
[]
[]
[1980]
[]
[]
[]
[]
[]
[2040]
[]
[]
[2070]
[]
[2090]
[]
[]
[2120]
[]
[]
[]
[]
[]
[2180]
[]
[]
[]
[2220]
[]
[]
[]
[]
[]
[]
[2290]
[]
[]
[]
[]
[]
[]
[2360]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[2460]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[2770]
[]
[]
[]
[2810

In [17]:
"""
get the average feature vector to represent shot with it's frames features 
params:
 List: array of the shot cut frames number
Return:
 average feature vector represent each frame   
"""
def get_avg_features_shot(shot_cuts:np.array):
    
    return 


In [18]:
# # TODO: get_HSV_feature_frame
# """
# params : frames's path
# return : HSV hist for all frames
# """
# def get_HSV_feature_frame():
#     # Set the number of histogram bins
#     num_bins = 16
#     HSV_hists = []
#     frames_path = 'Dataset/frames'
#     # Loop through the frames and extract the color histograms
#     for fram_name in os.listdir(frames_path):

#         frame = cv2.imread(os.path.join(frames_path, fram_name))
#         # Convert the frame to the HSV color space
#         hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

#         # Compute the color histogram for each channel
#         hist_hue = cv2.calcHist([hsv_frame], [0], None, [num_bins], [0, 180])
#         hist_saturation = cv2.calcHist([hsv_frame], [1], None, [num_bins], [0, 255])
#         hist_value = cv2.calcHist([hsv_frame], [2], None, [num_bins], [0, 255])

#         # Concatenate the histograms into a single feature vector
#         hist = np.concatenate([hist_hue, hist_saturation, hist_value], axis=0)

#         # Normalize the histogram to have unit L2 norm
#         hist = cv2.normalize(hist, hist)
#         HSV_hists.append(hist)
#     return 